In [1]:
#!pip install yfinance # uncomment and run to install

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

from scipy.stats import pearsonr, spearmanr, kendalltau

import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [3]:
frequency = "1d"
start_time = "2018-01-01"
end_time = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')  # Set end_time to yesterday

1. **TLT (iShares 20+ Year Treasury Bond ETF)**: Invests in U.S. Treasury bonds with remaining maturities greater than 20 years, aiming to track the investment results of an index composed of U.S. Treasury bonds.

2. **IEF (iShares 7-10 Year Treasury Bond ETF)**: Seeks to track the investment results of an index composed of U.S. Treasury bonds with remaining maturities between 7 and 10 years.

3. **EZU (iShares MSCI Eurozone ETF)**: Provides exposure to large and mid-sized companies in Eurozone countries, tracking the MSCI EMU Index.

4. **CNYA (iShares MSCI China A ETF)**: Offers exposure to large and mid-cap Chinese equities in the A-shares market, tracking the MSCI China A Inclusion Index.

5. **EWJ (iShares MSCI Japan ETF)**: Tracks the investment results of an index composed of Japanese equities, representing large and mid-sized companies in Japan.

6. **IGLT.L (iShares Core UK Gilts UCITS ETF)**: Invests in UK government bonds (gilts), seeking to track the performance of an index composed of sterling-denominated UK government bonds.

7. **XBB.TO (iShares Core Canadian Universe Bond Index ETF)**: Offers exposure to Canadian government and corporate bonds, tracking the performance of the broad Canadian bond market.

8. **BOND.AX (PIMCO Australian Bond Index Fund)**: Provides diversified exposure to the Australian bond market, including government, semi-government, and corporate debt securities.

9. **INDA (iShares MSCI India ETF)**: Aims to track the investment results of an index composed of Indian equities, representing large and mid-sized companies in India.

10. **EWL (iShares MSCI Switzerland ETF)**: Tracks the investment results of an index composed of Swiss equities, representing the Swiss stock market.

11. **ERUS (iShares MSCI Russia ETF)**: Seeks to track the investment results of an index composed of Russian equities.

12. **EWZ (iShares MSCI Brazil ETF)**: Aims to track the investment results of an index composed of Brazilian equities, reflecting the performance of the Brazilian stock market.

13. **EWY (iShares MSCI South Korea ETF)**: Tracks the investment results of an index composed of South Korean equities, representing the South Korean stock market.

14. **ENZL (iShares MSCI New Zealand ETF)**: Seeks to track the investment results of an index composed of New Zealand equities.

15. **EWD (iShares MSCI Sweden ETF)**: Aims to track the investment results of an index composed of Swedish equities, representing the Swedish stock market.

16. **EWM (iShares MSCI Malaysia ETF)**: Seeks to track the investment results of an index composed of Malaysian equities, representing the Malaysian stock market.

In [4]:
# Define ETFs
etfs = ["TLT", "IEF", "EZU", "CNYA", "EWJ", "IGLT.L", "XBB.TO", "BOND.AX", "INDA", "EWL", "ERUS", "EWZ", "EWY", "ENZL", "EWD", "EWM"]

# Fetch ETF data
etf_data = {etf: yf.Ticker(etf).info for etf in etfs}

# Retrieve AUM or market cap
aum = {etf: data.get('totalAssets', 0) for etf, data in etf_data.items()}  

# Calculate total AUM
total_aum = sum(aum.values())

# Calculate weightings
weights = {etf: value / total_aum for etf, value in aum.items()}

print(weights)

{'TLT': 0.38885760822225623, 'IEF': 0.23733374117905118, 'EZU': 0.05512229582981665, 'CNYA': 0.0018691532124829246, 'EWJ': 0.11149735828661055, 'IGLT.L': 0.0, 'XBB.TO': 0.05474578961840785, 'BOND.AX': 0.00031229276775030873, 'INDA': 0.05383947242196183, 'EWL': 0.010350169684085225, 'ERUS': 4.502729743237624e-05, 'EWZ': 0.047789691310546986, 'EWY': 0.03259198028276192, 'ENZL': 0.000986829085515959, 'EWD': 0.002654322728686626, 'EWM': 0.002004268072633402}


In [5]:
# Existing tickers and ETF tickers (& BTC, ETH in USD)
tickers = ["^IRX", "^TNX", "^TYX", "SPY", "QQQ"]
etf_tickers = etfs  
tickers.extend(etf_tickers)
crypto_tickers = ['BTC-USD']
tickers.extend(crypto_tickers)

# Retrieve historical data
historical_data = {}
for ticker in tickers:
    historical_data[ticker] = yf.download(ticker, start=start_time, end=end_time, interval=frequency)['Close']

# Combine and weight data
combined_data = pd.DataFrame()
for ticker, data in historical_data.items():
    if ticker in weights:  # Check if ticker is in ETFs list
        combined_data[ticker] = data * weights[ticker]
    else:  # Use the data as is for other tickers
        combined_data[ticker] = data

# Calculate the Global Liquidity Index as the sum of all (weighted and non-weighted) columns, excluding crypto tickers
combined_data['Global_Liquidity_Index'] = combined_data.drop(columns=crypto_tickers, errors='ignore').sum(axis=1)

# Count NaN values in each column
nan_count_per_column = combined_data.isna().sum()
print("NaN count per column:\n", nan_count_per_column)

# Total NaN values in the DataFrame
total_nan_count = combined_data.isna().sum().sum()
print("Total NaN values in the DataFrame:", total_nan_count)

combined_data.dropna(inplace=True)

# combined_data now contains the weighted Global Liquidity Index and individual ticker data

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [6]:
df = combined_data[['Global_Liquidity_Index', 'SPY', 'BTC-USD']].copy() # truncate df for dune upload

# Calculate the first difference for the specified columns
df['GLI_diff'] = df['Global_Liquidity_Index'].diff()
df['SPY_diff'] = df['SPY'].diff()
df['BTC_diff'] = df['BTC-USD'].diff()

df.dropna(inplace=True) # drop missing value from first differencing  
df

,Global_Liquidity_Index,SPY,BTC-USD,GLI_diff,SPY_diff,BTC_diff
Date,,,,,,
2018-01-03,529.168215,270.470001,15201.000000,3.613578,1.700012,218.900391
2018-01-04,530.703634,271.609985,15599.200195,1.535419,1.139984,398.200195
2018-01-05,534.124869,273.420013,17429.500000,3.421235,1.810028,1830.299805
2018-01-08,535.250383,273.920013,15170.099609,1.125514,0.500000,-2259.400391
2018-01-09,535.259241,274.540009,14595.400391,0.008857,0.619995,-574.699219
...,...,...,...,...,...,...
2023-12-22,974.601569,473.649994,43997.902344,1.416192,0.949982,128.750000
2023-12-27,981.543728,476.510010,43442.855469,6.942159,2.860016,-555.046875
2023-12-28,981.290662,476.690002,42627.855469,-0.253066,0.179993,-815.000000


In [7]:
df.to_csv('btc_global_liquidity_index.csv') 

In [8]:
for column in df.columns:
    # Perform Augmented Dickey-Fuller test:
    print(f'Testing for stationarity in series: {column}')
    result = adfuller(df[column].dropna())  # dropna() handles missing values
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    # Interpretation
    if result[1] > 0.05:
        print(f"The series {column} is likely non-stationary (fail to reject H0)\n")
        print("___________________________________________________________________")
    else:
        print(f"The series {column} is likely stationary (reject H0)\n")
        print("___________________________________________________________________")

Testing for stationarity in series: Global_Liquidity_Index


NameError: name 'adfuller' is not defined

In [ ]:
btc = df['BTC-USD']
gli = df['Global_Liquidity_Index']  
spy = df['SPY'] # benchmark 

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def johansen_cointegration_test(ts1, ts2):
    """
    Perform the Johansen cointegration test on two time series.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.

    Returns:
    None
    """
    # Combine the series into a 2D array
    ts_combined = np.column_stack([ts1, ts2])

    # Perform the Johansen cointegration test
    result = coint_johansen(ts_combined, det_order=0, k_ar_diff=1)

    # The trace statistic and the maximum eigenvalue statistic
    trace_stat = result.lr1
    max_eig_stat = result.lr2

    # Critical values at 90%, 95%, and 99% confidence intervals
    trace_crit_vals = result.cvt
    max_eig_crit_vals = result.cvm

    # Print results
    print("Johansen Cointegration Test Results:")
    print("------------------------------------")
    for idx, (trace, max_eig, trace_crit, max_eig_crit) in enumerate(zip(trace_stat, max_eig_stat, trace_crit_vals, max_eig_crit_vals)):
        print(f"Result for the Hypothesis of {idx} cointegrating relations:")
        print(f"Trace Statistic: {trace:.2f}, Critical Values (90%, 95%, 99%): {trace_crit}")
        print(f"Max-Eigen Statistic: {max_eig:.2f}, Critical Values (90%, 95%, 99%): {max_eig_crit}")
        print("")

    # Interpretation of the test
    print("Interpretation:")
    print("---------------")
    print("If the Trace and/or Max-Eigen statistics are greater than the 95% critical values, the null")
    print("hypothesis of 'no cointegration' is rejected, indicating cointegration relations.")

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def johansen_cointegration_test(ts1, ts2, confidence_level=0.05):
    """
    Perform the Johansen cointegration test on two time series and provide an interpretation.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.
    confidence_level : float
        The confidence level for determining cointegration (default is 0.05).

    Returns:
    None
    """
    # Combine the series into a 2D array
    ts_combined = np.column_stack([ts1, ts2])

    # Perform the Johansen cointegration test
    result = coint_johansen(ts_combined, det_order=0, k_ar_diff=1)

    # The trace statistic and the maximum eigenvalue statistic
    trace_stat = result.lr1
    max_eig_stat = result.lr2

    # Critical values at chosen confidence level (90%, 95%, 99%)
    crit_val_index = {0.10: 0, 0.05: 1, 0.01: 2}[confidence_level]
    trace_crit_vals = result.cvt[:, crit_val_index]
    max_eig_crit_vals = result.cvm[:, crit_val_index]

    # Print results and interpretation
    print("Johansen Cointegration Test Results and Interpretation:")
    print("-------------------------------------------------------")
    for idx, (trace, max_eig, trace_crit, max_eig_crit) in enumerate(zip(trace_stat, max_eig_stat, trace_crit_vals, max_eig_crit_vals)):
        print(f"Result for the Hypothesis of {idx} cointegrating relations:")
        print(f"Trace Statistic: {trace:.2f}, Critical Value: {trace_crit:.2f}")
        print(f"Max-Eigen Statistic: {max_eig:.2f}, Critical Value: {max_eig_crit:.2f}")

        if trace > trace_crit and max_eig > max_eig_crit:
            print("Both Trace and Max-Eigen statistics are above the critical values, suggesting cointegration.")
        elif trace > trace_crit or max_eig > max_eig_crit:
            print("One of the statistics is above the critical value, suggesting possible cointegration.")
        else:
            print("Neither statistic is above the critical value, suggesting no cointegration.")
        print("")

# Correlation

In [9]:
corr_df = df.copy()

In [10]:
def calculate_correlations(ts1, ts2):
    """
    Calculate the Pearson, Spearman, and Kendall Tau correlation coefficients along with their p-values for two time series.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.

    Returns:
    dict
        A dictionary containing the Pearson, Spearman, and Kendall Tau correlation coefficients and their p-values.
    """
    pearson_corr, pearson_p = pearsonr(ts1, ts2)
    spearman_corr, spearman_p = spearmanr(ts1, ts2)
    kendall_corr, kendall_p = kendalltau(ts1, ts2)

    return {
        'Pearson': {'Coefficient': pearson_corr, 'P-Value': pearson_p},
        'Spearman': {'Coefficient': spearman_corr, 'P-Value': spearman_p},
        'Kendall Tau': {'Coefficient': kendall_corr, 'P-Value': kendall_p}
    }

In [11]:
gli_btc_corr = calculate_correlations(corr_df['Global_Liquidity_Index'], corr_df['BTC-USD'])
gli_btc_corr

{'Pearson': {'Coefficient': 0.8693334622326608, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.9109390889880306, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.733494160147782, 'P-Value': 0.0}}

In [12]:
i1_gli_btc_corr = calculate_correlations(corr_df['GLI_diff'], corr_df['BTC_diff'])
i1_gli_btc_corr

{'Pearson': {'Coefficient': 0.2707416127256846,
  'P-Value': 1.4091913565896897e-25},
 'Spearman': {'Coefficient': 0.2642358484218297,
  'P-Value': 2.129460142036639e-24},
 'Kendall Tau': {'Coefficient': 0.1813681248351818,
  'P-Value': 6.763503970929212e-25}}

In [13]:
gli_spy_corr = calculate_correlations(corr_df['Global_Liquidity_Index'], corr_df['SPY'])
gli_spy_corr

{'Pearson': {'Coefficient': 0.989583982577496, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.9896298891645711, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.9232025995935197, 'P-Value': 0.0}}

In [14]:
i1_gli_spy_corr = calculate_correlations(corr_df['GLI_diff'], corr_df['SPY_diff'])
i1_gli_spy_corr

{'Pearson': {'Coefficient': 0.9764064383179702, 'P-Value': 0.0},
 'Spearman': {'Coefficient': 0.9707538911082162, 'P-Value': 0.0},
 'Kendall Tau': {'Coefficient': 0.8633657607478469, 'P-Value': 0.0}}

In [15]:
def create_correlation_dataframe(dicts, names):
    """
    Create a pandas DataFrame from a list of dictionaries containing correlation coefficients, with named rows.

    Parameters:
    dicts : list of dict
        List of dictionaries with correlation data.
    names : list of str
        List of names for each time series pair.

    Returns:
    pd.DataFrame
        A DataFrame with correlation coefficients and named rows.
    """
    # Prepare data for DataFrame
    data = {name: {metric: d[metric]['Coefficient'] for metric in ['Pearson', 'Spearman', 'Kendall Tau']} for name, d in zip(names, dicts)}

    # Create DataFrame from the data
    corr_df = pd.DataFrame(data).T

    return corr_df

# Names for each time series pair
names = ["GLI-BTC", "I1-GLI-BTC", "GLI-SPY", "I1-GLI-SPY"]

# Creating the DataFrame
corr_df = create_correlation_dataframe([gli_btc_corr, i1_gli_btc_corr, gli_spy_corr, i1_gli_spy_corr], names)

# Displaying the DataFrame
corr_df.index.name = 'time_series_pairs'
corr_df.reset_index(inplace=True)
corr_df

,time_series_pairs,Pearson,Spearman,Kendall Tau
0,GLI-BTC,0.869333,0.910939,0.733494
1,I1-GLI-BTC,0.270742,0.264236,0.181368
2,GLI-SPY,0.989584,0.989630,0.923203
3,I1-GLI-SPY,0.976406,0.970754,0.863366


In [16]:
corr_df.to_csv('btc_global-liquidity_corr.csv')

# Cointegration

In [17]:
def johansen_cointegration_test(ts1, ts2, confidence_level=0.05):
    """
    Perform the Johansen cointegration test on two time series and provide an interpretation.

    Parameters:
    ts1 : Pandas Series
        The first time series.
    ts2 : Pandas Series
        The second time series.
    confidence_level : float
        The confidence level for determining cointegration (default is 0.05).

    Returns:
    None
    """
    # Combine the series into a 2D array
    ts_combined = np.column_stack([ts1, ts2])

    # Perform the Johansen cointegration test
    result = coint_johansen(ts_combined, det_order=0, k_ar_diff=1)

    # The trace statistic and the maximum eigenvalue statistic
    trace_stat = result.lr1
    max_eig_stat = result.lr2

    # Critical values at chosen confidence level (90%, 95%, 99%)
    crit_val_index = {0.10: 0, 0.05: 1, 0.01: 2}[confidence_level]
    trace_crit_vals = result.cvt[:, crit_val_index]
    max_eig_crit_vals = result.cvm[:, crit_val_index]

    # Print results and interpretation
    print("Johansen Cointegration Test Results and Interpretation:")
    print("-------------------------------------------------------")
    for idx, (trace, max_eig, trace_crit, max_eig_crit) in enumerate(zip(trace_stat, max_eig_stat, trace_crit_vals, max_eig_crit_vals)):
        print(f"Result for the Hypothesis of {idx} cointegrating relations:")
        print(f"Trace Statistic: {trace:.2f}, Critical Value: {trace_crit:.2f}")
        print(f"Max-Eigen Statistic: {max_eig:.2f}, Critical Value: {max_eig_crit:.2f}")

        if trace > trace_crit and max_eig > max_eig_crit:
            print("Both Trace and Max-Eigen statistics are above the critical values, suggesting cointegration.")
        elif trace > trace_crit or max_eig > max_eig_crit:
            print("One of the statistics is above the critical value, suggesting possible cointegration.")
        else:
            print("Neither statistic is above the critical value, suggesting no cointegration.")
        print("")

In [18]:
gli = df['Global_Liquidity_Index']
btc = df['BTC-USD']
spy = df['SPY']

In [19]:
johansen_cointegration_test(gli, btc)

Johansen Cointegration Test Results and Interpretation:
-------------------------------------------------------
Result for the Hypothesis of 0 cointegrating relations:
Trace Statistic: 10.25, Critical Value: 15.49
Max-Eigen Statistic: 9.91, Critical Value: 14.26
Neither statistic is above the critical value, suggesting no cointegration.

Result for the Hypothesis of 1 cointegrating relations:
Trace Statistic: 0.34, Critical Value: 3.84
Max-Eigen Statistic: 0.34, Critical Value: 3.84
Neither statistic is above the critical value, suggesting no cointegration.



In [20]:
johansen_cointegration_test(gli, spy)

Johansen Cointegration Test Results and Interpretation:
-------------------------------------------------------
Result for the Hypothesis of 0 cointegrating relations:
Trace Statistic: 9.13, Critical Value: 15.49
Max-Eigen Statistic: 6.85, Critical Value: 14.26
Neither statistic is above the critical value, suggesting no cointegration.

Result for the Hypothesis of 1 cointegrating relations:
Trace Statistic: 2.29, Critical Value: 3.84
Max-Eigen Statistic: 2.29, Critical Value: 3.84
Neither statistic is above the critical value, suggesting no cointegration.

